In [27]:
import pandas as pd
import os
import json
from tqdm import tqdm
pd.set_option('max_colwidth', 400)

In [28]:
import pandas as pd
from scipy import spatial
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from scipy import stats
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, LoggingHandler, util, models, evaluation, losses, InputExample

# Leitura da entrada

In [29]:
def transform_json(diario_json):
    dict_list =  []
    for key, value in json.loads(diario_json).items():
        for v in value:
            v["origin"] = key
            dict_list.append(v)
    return pd.DataFrame(dict_list)

In [30]:
dfs = []
directory = "../saida_segmentacao/AMM/"
for filename in tqdm(list(os.listdir(directory))):
    with open(directory + filename, "r") as jsonfile:
        diario_json = jsonfile.read()
        dfs.append(transform_json(diario_json))

100%|█████████████████████████████████████████████████████████████████████████████| 1640/1640 [00:20<00:00, 81.80it/s]


In [32]:
diario_df = pd.concat(dfs)
diario_df.sample(3)

,titulo,subtitulo,materia,publicador,id,origin
253,SECRETARIA MUNICIPAL DE ASSISTÊNCIA SOCIAL,RESULTADO PONTUAÇÃO DE TÍTULOS EDITAL SMAS Nº 005/2019,Nº 01 02 03 Maria Luiza Theodoro 04 Meirielle Badaró Silva 05 06Nome Bárbara de Abreu Procópio Vivian Fontanella de Melo Pontuação 10 10 2 2 2 2 Luana Rodrigues Pires Lotte Iano Bittencourt de Souza 07 08 09 10 Marisa Aparecida de Oliveira Fajardo 11 12 Maria Marta Severo Jaqueline Dutra de Castro 13 Ivone de Moraes Camilo 14 Carmen Aparecida Medeiros Netto 15 Regina Barbosa Vargas 16 Fernanda...,Tatiane Bonini Cosine,98643BA4,PREFEITURA DE LEOPOLDINA
96,PREFEITURA MUNICIPAL DE DIVINÓPOLIS,,"SECRETARIA MUNICIPAL DE ADMINISTRAÇÃO,ORÇAMENTO E INFORMAÇÃO O Secretário de Administração, Orçamento e Informação, Sr. Gilberto Tavares Machado, no uso de suas atribuições, CANCELA a prorrogação da jornada de trabalho (extensão de carga horária) de 02 (duas) horas por dia à servidora IVONETE DOLHER, matriculada sob o nº 0202021-4, detentora do cargo efetivo de Servente Escolar, com efeitos re...",Daniel Felipe da Costa,226A1227,PREFEITURA DE DIVINÓPOLIS
49,SERVIÇO DE PESSOAL,,"PORTARIA Nº 2694/2014 CONCEDE LICENÇAMATERNIDADE À LILIANE REIS ASSIS A Prefeita Municipal de Desterro do Melo, no uso de suas atribuições legais e, em conformidade com o inciso VI do art. 66 da Lei Orgânica Municipal e, em especial com o Art. 7º, inciso XVIII da Constituição Federal de 1988, RESOLVE: Art. 1º Fica concedida Licença Maternidade à servidora Liliane Reis Assis, ocupante do cargo ...",Danilo Amaral Cimino,421D3AF4,MUNICÍPIO DE DESTERRO DO MELO


# Criação do dataset

In [33]:
def transform_df(diario_df):
    diario_df["query"] = diario_df.apply(lambda row:row["origin"] +" " +  row["titulo"] +" " + row["subtitulo"],axis = 1)
    diario_df["doc"] = diario_df["materia"]
    return diario_df[["query", "doc"]]

In [34]:
train_df = transform_df(diario_df)
train_df

,query,doc
0,ASSOCIAÇÃO DOS MUNICIPIOS DE BAIXA MOGIANA EDITAIS,"TERMO DE HOMOLOGAÇÃO E ADJUDICAÇÃO -PROCESSO LICITATÓRIO Nº: 03/2014 - PREGÃO Nº: 01/2014 Pelo presente termo, levando em consideração o julgamento do Pregoeiro, que examinou as propostas referentes ao processo acima identificado, apontando como propostas mais vantajosas à Associação as apresentadas pelas empresas: CSC CONSTRUTORA SIQUEIRA CARDOSO LTDA, estabelecida na Rua dos Caetés, 285, CEP..."
1,ASSOCIAÇÃO DOS MUNICIPIOS DE BAIXA MOGIANA Expediente:,Associação Mineira de Municípios – AMM - MG Diretoria 2011/2013 Presidente - Ângelo José Roncalli de Freitas 1º Vice-Presidente - Acácio Mendes de Andrade 2º Vice-Presidente - José Milton de Carvalho Rocha 3º Vice-Presidente - Marco Antônio de Andrade 1º Secretário - Aurelio Cezar Donadia Ferreira 2º Secretário - Eduardo Antônio Carvalho 1º Tesoureiro - José Sacido Bacia Neto 2º Tesoureiro - M...
2,ASSOCIAÇÃO DOS MUNICIPIOS DE BAIXA MOGIANA SETOR DE COMPRAS E LICITAÇÕES,"PREGÃO PRESENCIAL 03/2015 A Câmara Municipal de Pará de Minas, com sede à Avenida Presidente Vargas, 1935 nesta cidade de Pará de Minas/MG, por intermédio de sua Pregoeira, torna público, para conhecimento dos interessados, que fará realizar licitação com as seguintes características: MODALIDADE: Pregão Presencial Nº03/2015 TIPO: Menor preço por lote OBJETO: REGISTRO DE PREÇO para futuras e ev..."
3,CÂMARA MUNICIPAL DE RIO PARANAÍBA CÂMARA MUNICIPAL DE RIO PARANAÍBA,"PREGÃO PRESENCIAL Nº 01/2015. AVISO DE LICITAÇÃO PROCESSO LICITATÓRIO Nº 02/2015 PREGÃO PRESENCIAL Nº 01/2015 OBJETO:Constitui objeto da presente licitação: Aquisição parcelada de combustível mediante requisição periódica, com especificações contidas no Anexo I - Termo de Referência do Edital. ·APRESENTAÇÃO PARA CREDENCIAMENTO DOS LICITANTES:Dia 09/02/2015, às 14h (quatorze horas). ·ABERTURA D..."
4,PREFEITURA DE ALVORADA DE MINAS LICITAÇÃO,"EDITAL PREGAO PRESENCIAL 05/2015 A Prefeitura Municipal de Alvorada de Minas torna público que fará realizar Processo Licitatório Pregão Presencial n.º 0005/2014. Objeto: REFERENTE A LOCACAO DE TRES TRATORES AGRICOLAS COM IMPLEMENTO ( GRADE, ENSILADEIRA E CARROÇA) PARA ATENDIMENTO AO PROGRAMA DE INCENTIVO AO PEQUENO PRODUTOR RURAL CONFORME LEI MUNICIPAL 832/2013 PARA O ANO DE 2015.. Entrega do..."
...,...,...
901,PREFEITURA MUNICIPAL DE VISCONDE DO RIO BRANCO DEPARTAMENTO DE CONVÊNIOS EXERCÍCIO DE 2020,EXTRATO DE PUBLICAÇÃO DISPENSA DE CHAMAMENTO PÚBLICO CELEBRAÇÃO DE TERMO DE FOMENTO N°. 002/2020 PARTES: MUNICÍPIO DE VISCONDE DO RIO BRANCO – MINAS GERAIS e as OBRAS SOCIAIS DA PAROQUIA DE SÃO JOÃO BATISTA DE VISCONDE DO RIO BRANCO – CENTRO SOCIAL SÃO FRANCISCO DE ASSIS; CENTRO SOCIAL SANTA CLARA; LAR SÃO JOÃO BATISTA (LAR DOS IDOSOS). OBJETO: Firmar TERMO DE FOMENTO cujo objetivo é o repasse...
902,PREFEITURA MUNICIPAL DE VISCONDE DO RIO BRANCO DEPARTAMENTO DE LICITAÇÃO E CONTRATOS EXTRATO DE AVISO DE PENALIDADE,"AVISO DE PENALIDADE – PREGÃO 058/2019 – PROCESSO LICITATÓRIO 108/2019 – ÓRGÃO GERENCIADOR: PREFEITURA MUNICIPAL DE VISCONDE DO RIO BRANCO – FORNECEDOR REGISTRADO: POLO COMERCIAL EIRELI– 1) Aplicação de advertência à empresa de não mais participar de certames dos quais não possa cumprir com as obrigações assumidas e conforme as especificações do Edital, sob pena de aplicação de penalidade mais ..."
903,SAAE - SERVIÇO AUTÔNOMO DE ÁGUA E ESGOTO DE PIRAPORA SAAE PIRAPORA-MG ERRATA CONTRATO 12/2020,"Serviço Autônomo de Água e Esgoto de Pirapora-MG. Erata: Na publicação realizada no dia 03/01/2020, pág. 201, onde se lê: “Extrato de Contrato. Contrato 10/2020. Contratado: Banco do Nordeste do Brasil SA – CNPJ 07.237.373/0077-28”, leia-se: “Extrato de Contrato. Contrato 12/2020. Contratado: Banco do Nordeste do Brasil SA – CNPJ 07.237.373/0077-28”.ALESSANDRA BARBOSA ROCHA VELOSO Licitações e..."
904,SAAE - SERVIÇO AUTÔNOMO DE ÁGUA E ESGOTO DE PIRAPORA SAAE PIRAPORA-MG CONTRATO 13/2020,"Serviço Autônomo de Água e Esgoto de Pirapora-MG. Extrato de 

# Treinamento

In [35]:
def average_similarity():
    false_pairs, true_pairs = 0, 0
    SAMPLE_SIZE = 1000
    df_sample = train_df.sample(SAMPLE_SIZE, random_state=42)
    for i in tqdm(range(1, SAMPLE_SIZE)):
      row, neg = df_sample.iloc[i], df_sample.iloc[i - 1]

      output = model.encode([row["query"], row["doc"], neg["doc"]]), 

      true_pairs += 1 - spatial.distance.cosine(output[0][0], output[0][1])
      false_pairs += 1 - spatial.distance.cosine(output[0][0], output[0][2])

    print("Média de similaridade dos positivos: ", true_pairs / SAMPLE_SIZE)
    print("Média de similaridade dos negativos: ", false_pairs / SAMPLE_SIZE)

In [40]:
from sentence_transformers import SentenceTransformer, models
from torch import nn

word_embedding_model = models.Transformer('neuralmind/bert-base-portuguese-cased', max_seq_length=150)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [41]:
average_similarity()

100%|███████████████████████████████████████████████████████████████████████████████| 999/999 [00:22<00:00, 44.09it/s]

Média de similaridade dos positivos:  0.7687452844977379
Média de similaridade dos negativos:  0.7516179756540805


In [42]:
examples = []
SAMPLE_SIZE = 20000
for i in tqdm(range(1, len(train_df.sample(SAMPLE_SIZE, random_state=24))), position=0, leave=True, total=SAMPLE_SIZE):
  row, neg = train_df.iloc[i], train_df.iloc[i - 1]
  examples.append(InputExample(texts=[row["query"], row["doc"], neg["doc"]]))

100%|████████████████████████████████████████████████████████████████████████▉| 19999/20000 [00:03<00:00, 6373.71it/s]


In [43]:
dataloader = DataLoader(examples, shuffle=True, batch_size=8)
train_loss = losses.TripletLoss(model)
model.fit(train_objectives=[(dataloader, train_loss)], epochs=1, warmup_steps=50)

In [44]:
average_similarity()

100%|███████████████████████████████████████████████████████████████████████████████| 999/999 [00:23<00:00, 42.14it/s]

Média de similaridade dos positivos:  0.6971365534086944
Média de similaridade dos negativos:  0.23703911202040034
